<a href="https://colab.research.google.com/github/Suchitra-V31/Deep_Learning_Projects/blob/main/Cat_vs_Dog_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this dataset we are going to train the model with cats and dogs images and make the model to predict whether it is a cat or dog.

In [ ]:
#import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import tensorflow

In [ ]:
#training and image processing
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
#cnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import RMSprop


In [ ]:
import cv2

In [ ]:
Image_height=128
Image_width=128
Image_size=(Image_width,Image_height)
Image_channels=3

In [ ]:

filenames=os.listdir('/content/drive/MyDrive/Datasets/Cat vs Dog/train') # train data

In [ ]:
categories=[]
data=[]
for f_name in filenames:
  category=f_name.split('.')[0]
  if category=='dog':
    categories.append(1)
  else:
    categories.append(0)
    

In [ ]:
df=pd.DataFrame({'filenames':filenames,'categories':categories})

In [ ]:
df.head()

,filenames,categories
0,cat.9113.jpg,0
1,cat.909.jpg,0
2,cat.9093.jpg,0
3,cat.9075.jpg,0
4,cat.9078.jpg,0


In [ ]:
df.tail()

,filenames,categories
24365,dog.1033.jpg,1
24366,dog.10333.jpg,1
24367,dog.10338.jpg,1
24368,dog.10331.jpg,1
24369,dog.10332.jpg,1


**Design Neural Network Architecture**

In [ ]:
model=Sequential()
model.add(Conv2D(64,(3,3),activation='relu',padding='Same',input_shape=(Image_width,Image_height,Image_channels)))
model.add(BatchNormalization())

model.add(Conv2D(128,(3,3),activation='relu',padding='Same',input_shape=(Image_width,Image_height,Image_channels)))
model.add(Conv2D(128,(3,3),activation='relu',padding='Same',input_shape=(Image_width,Image_height,Image_channels)))
model.add(Conv2D(128,(3,3),activation='relu',padding='Same',input_shape=(Image_width,Image_height,Image_channels)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(2,activation='sigmoid'))

**Compile Model**

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 524288)            0

In [ ]:
df['categories']=df['categories'].replace({0:'cat',1:'dog'})

In [ ]:
train_df,valid_df=train_test_split(df,test_size=0.33,random_state=0) #split the train data into train/valid

In [ ]:
train_df=train_df.reset_index(drop=True)
valid_df=valid_df.reset_index(drop=True)

In [ ]:
total_train=train_df.shape[0]
total_train

16327

In [ ]:
total_valid=valid_df.shape[0]
total_valid

8043

In [ ]:
train_df.head()

,filenames,categories
0,cat.2802.jpg,cat
1,cat.11834.jpg,cat
2,dog.1120.jpg,dog
3,dog.3089.jpg,dog
4,dog.12178.jpg,dog


In [ ]:
valid_df.head()

,filenames,categories
0,cat.9926.jpg,cat
1,dog.12263.jpg,dog
2,dog.1314.jpg,dog
3,cat.3589.jpg,cat
4,dog.12116.jpg,dog


**Use Data Augmentation to get multiple images**

In [ ]:
batch_size=32
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

In [ ]:
train_generator=train_datagen.flow_from_dataframe(train_df,'/content/drive/MyDrive/Datasets/Cat vs Dog/train',
                                                  x_col='filenames',
                                                  y_col='categories',
                                                  batch_size=32,
                                                  target_size=Image_size,
                                                  class_mode='categorical')

Found 16327 validated image filenames belonging to 2 classes.


In [ ]:
valid_generator=train_datagen.flow_from_dataframe(valid_df,'/content/drive/MyDrive/Datasets/Cat vs Dog/train',
                                                  x_col='filenames',
                                                  y_col='categories',
                                                  batch_size=32,
                                                  target_size=Image_size,
                                                  class_mode='categorical')

Found 8043 validated image filenames belonging to 2 classes.


**Train!!!!**

In [ ]:
epochs=5
history=model.fit_generator(train_generator,
                            steps_per_epoch=total_train//batch_size,
                            validation_data=valid_generator,
                            validation_steps=total_valid//batch_size,
                            epochs=epochs)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
510/510 [==============================] - 5976s 12s/step - loss: 0.7438 - accuracy: 0.5561 - val_loss: 0.9597 - val_accuracy: 0.5173
Epoch 2/5
510/510 [==============================] - 314s 614ms/step - loss: 0.6483 - accuracy: 0.6289 - val_loss: 0.6703 - val_accuracy: 0.6155
Epoch 3/5
510/510 [==============================] - 312s 612ms/step - loss: 0.5939 - accuracy: 0.6795 - val_loss: 0.6636 - val_accuracy: 0.6470
Epoch 4/5
510/510 [==============================] - 310s 607ms/step - loss: 0.5710 - accuracy: 0.6976 - val_loss: 0.6456 - val_accuracy: 0.6718
Epoch 5/5
510/510 [==============================] - 313s 614ms/step - loss: 0.5471 - accuracy: 0.7185 - val_loss: 0.6468 - val_accuracy: 0.6754


We could see that our model has predicted with 71% accuracy.